In [4]:
# Trade Filter using Statistical Learning (No sklearn)

import pandas as pd
import numpy as np


In [5]:
data = pd.read_csv("../results/strategy_results.csv")

data.head()


,datetime,open_spot,high_spot,low_spot,close_spot,volume_spot,open_fut,high_fut,low_fut,close_fut,...,spot_return,futures_return,futures_basis,pcr_oi,pcr_volume,avg_iv,regime,signal,strategy_return,equity_curve
0,2024-01-01 00:10:00,18001.006138,18013.734743,17998.845623,18007.006533,105033,18001.006138,18013.734743,17998.845623,18011.563403,...,0.000311,-0.000174,0.000253,0.638465,1.894637,0.200632,1,1.0,0.000311,1.000311
1,2024-01-01 00:15:00,18002.529168,18016.658683,17995.751955,18000.069656,173324,18002.529168,18016.658683,17995.751955,18006.700648,...,-0.000385,-0.000270,0.000368,1.631559,1.082327,0.236436,0,1.0,-0.000385,0.999925
2,2024-01-01 00:20:00,18002.295015,18002.926738,17986.243301,17992.912251,115689,18002.295015,18002.926738,17986.243301,18013.520479,...,-0.000398,0.000379,0.001145,0.975458,1.104844,0.196192,-1,0.0,-0.000000,0.999925
3,2024-01-01 00:25:00,18002.060878,18020.785123,17990.619905,18005.159435,209756,18002.060878,18020.785123,17990.619905,18010.871912,...,0.000681,-0.000147,0.000317,1.541274,1.234718,0.207352,1,-1.0,-0.000681,0.999245
4,2024-01-01 00:30:00,18003.640091,18004.679516,17993.386736,18000.463279,100066,18003.640091,18004.679516,17993.386736,18006.905177,...,-0.000261,-0.000220,0.000358,0.704347,0.656493,0.164896,0,1.0,-0.000261,0.998984


In [6]:
data["profitable"] = (data["strategy_return"] > 0).astype(int)

data["profitable"].value_counts()


profitable
0    882
1    116
Name: count, dtype: int64

In [7]:
data["ema_gap"] = abs(data["ema_5"] - data["ema_15"])


In [8]:
threshold = data["ema_gap"].quantile(0.6)

data["ml_signal"] = np.where(
    (data["ema_gap"] > threshold) & (data["regime"] != 0),
    1,
    0
)

data["ml_signal"].value_counts()


ml_signal
0    744
1    254
Name: count, dtype: int64

In [9]:
data["ml_strategy_return"] = data["strategy_return"] * data["ml_signal"]

data["ml_equity_curve"] = (
    1 + data["ml_strategy_return"]
).cumprod()


In [10]:
base_return = data["equity_curve"].iloc[-1] - 1
ml_return = data["ml_equity_curve"].iloc[-1] - 1

print("Base Strategy Return:", round(base_return, 2))
print("ML-Filtered Strategy Return:", round(ml_return, 2))


Base Strategy Return: -0.15
ML-Filtered Strategy Return: -0.0


In [11]:
data.to_csv("../results/ml_strategy_results.csv", index=False)
print("Statistical ML filter completed")


Statistical ML filter completed
